# Baby EmoLLM

- 本教程分为两个部分：模型微调和RAG
- 这里只做简单的流程演示，让大家明白模型微调和RAG基本原理
- 本项目使用的微调框架为XTuner

## 模型微调

### 环境安装

In [ ]:
# ! pip install transformers
# ! pip install torch
# ! pip install datasets
# ! pip install peft

### 导入相关包

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer
from datasets import load_dataset

### 模型下载

In [ ]:
# from modelscope.hub.snapshot_download import snapshot_download
# snapshot_download(model_id="Shanghai_AI_Laboratory/internlm2-chat-1_8b", cache_dir="./models")

### 加载数据集

- 本项目的所有数据集存储在`./datasets`文件夹下
- 数据集的介绍请参考`./datasets/README.md`
- 微调数据集的构建指南请参考`./generate_data/tutorial.md`
- 本实验的微调数据集为`./datasets/processed_single_turn_dataset_1.json`

In [2]:
ds= load_dataset("json",  data_files="./datasets/processed_single_turn_dataset_1.json", split="train")
ds

Dataset({
    features: ['conversation'],
    num_rows: 14041
})

### 加载模型

- 本实验的模型采用的模型为`internlm2-chat-1_8b`
- 你也可以将模型更换为其他模型
- 微调策略采用的是`QLoRA`

In [3]:
model_dir = "./Shanghai_AI_Laboratory/internlm2-chat-1_8b"
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_dir, low_cpu_mem_usage=True, 
                                            torch_dtype=torch.bfloat16,
                                            device_map="auto",
                                            load_in_4bit=True,
                                            bnb_4bit_compute_dtype=torch.bfloat16,
                                            bnb_4bit_quant_type="nf4",
                                            bnb_4bit_use_double_quant=True,
                                            trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### 数据预处理

- 构造数据处理函数，将数据集中的对话转换为模型的输入格式
- 当我们使用Xtuner进行微调时，数据处理会自动进行，无需手动处理
- 我们只需要提供处理好的对话数据即可

In [5]:
def process_func(example):
    MAX_LENGTH = 512
    input_ids, attention_mask, labels = [], [], []
    instruction = example["conversation"][0]["input"]
    instruction = model.build_inputs(tokenizer, instruction, history=[], meta_instruction=example["conversation"][0]["system"])  
    response = tokenizer(example["conversation"][0]["output"], add_special_tokens=False)       

    
    input_ids = instruction["input_ids"][0].numpy().tolist() + response["input_ids"] + [tokenizer.eos_token_id]
    attention_mask = instruction["attention_mask"][0].numpy().tolist() + response["attention_mask"] + [1]
    labels = [-100] * len(instruction["input_ids"][0].numpy().tolist()) + response["input_ids"] + [tokenizer.eos_token_id]


    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]

    
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [6]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds 

Parameter 'function'=<function process_func at 0x7fefd48c6b00> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 14041
})

### 配置QLoRA

- QLoRA是一种高效的大模型微调策略
- 在微调时，我们可以选择对模型的部分参数进行微调
- 通过配置`LoraConfig`，我们可以指定微调的目标模块

In [9]:
from peft import LoraConfig, TaskType, get_peft_model
config = LoraConfig(task_type=TaskType.CAUSAL_LM, target_modules=["wqkv", "w1", "w2", "w3"])

In [10]:
# 查看QLoRA配置文件
print(config)

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules=['wqkv', 'w1', 'w2', 'w3'], lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None)


In [11]:
model = get_peft_model(model, config)  # 根据配置文件在原模型的基础上添加QLoRA模块

In [12]:
model.enable_input_require_grads()

In [13]:
model.print_trainable_parameters()

trainable params: 7,077,888 || all params: 1,896,187,904 || trainable%: 0.37326933607525004


 trainable params ：表示参与训练的参数量
 all params ： 模型的整个参数量
 trainable ： 参与训练的参数占整个参数的比例

### 配置训练参数

- 我们可以通过`TrainingArguments`来配置训练参数
- output_dir: QLoRA模型保存路径
- per_device_train_batch_size: 每个设备的训练batch_size
- gradient_accumulation_steps: 梯度累积步数
- logging_steps: 日志输出步数
- num_train_epochs: 训练轮数
- learning_rate: 学习率
- gradient_checkpointing: 梯度检查点
- optim: 优化器

In [14]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    logging_steps=10,
    num_train_epochs=1,
    learning_rate=3e-4,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit"
)

### 创建训练器

- model ： 加载了QLoRA适配器的model
- args : 训练的参数配置
- train_dataset ： 处理后的训练数据集
- data_collator ： 数据集规整器

In [15]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


### 模型训练

- 完成以上步骤后，我们可以开始训练模型
- 本项目所有的训练脚本都存储在`xtuner_config`文件夹下

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

### 模型评估

- 同时加载训练好的QLoRA模型和原始模型进行推理评估

In [16]:
from peft import PeftModel
p_model = PeftModel.from_pretrained(model, model_id="./chatbot/checkpoint-876")

In [18]:
p_model.eval()
print(p_model.chat(tokenizer, "我感觉我心理不健康", history=[])[0])

我很理解你对心理健康的关注。心理健康就像是一棵茂盛的树，需要我们细心呵护。就像树的根系一样，心理健康需要我们的积极思考、乐观的心态和有效的应对压力的能力。

有时候，我们可能会感到心理不健康，这可能是因为我们面临着一些挑战和压力。就像在暴风雨中，树可能会摇摇欲坠，但是只要我们坚守，它就会恢复健康。所以，当你感到心理不健康时，不要独自承受，寻求专业的帮助和支持，与他人分享你的感受，他们会给予你更多的力量和支持。

同时，我们也可以通过一些方法来改善心理健康。就像树需要阳光和水分一样，我们也需要找到适合自己的方式来放松自己。这可以包括运动、冥想、艺术创作、与朋友交流等等。这些活动可以帮助我们减轻压力，提升情绪，让我们重新获得内心的平静和健康。

最后，记住心理健康是一个长期的过程，而不是一蹴而就的结果。就像树需要时间成长一样，我们也需要耐心和持续的努力来改善自己的心理健康。如果你感觉心理不健康，不要犹豫寻求帮助，并相信自己可以重新获得内心的平衡和健康。


## RAG

- 本项目的RAG实现存储在`rag`文件夹下
- 这里只做最基本的RAG实现
- RAG的数据采用QA对形式，具体的数据生成方法请参考`scripts/qa_generation/README.md`

### 环境安装

In [ ]:
# ! pip install numpy
# ! pip install transformers
# ! pip install BCEmbedding
# ! pip install faiss-gpu
# ! pip install peft

### 加载数据

In [1]:
import json
import numpy as np
def data_process(dir):
    """读取jsonl文件，返回问题和答案的列表"""
    with open(dir, 'r', encoding='utf-8') as f:
        lines = f.readlines()   
        lines = [json.loads(line) for line in lines]
        questions = np.array([line['question'] for line in lines])
        answers = np.array([line['answer'] for line in lines])
        f.close()
    return questions, answers

questions, answers = data_process('./test.jsonl') # 这里传入你的数据集路径

In [2]:
print(questions[0])
print(answers[0])

《儿童发展心理学》的主要内容是什么？
《儿童发展心理学》主要探讨0一18岁个体心理发展的特征及影响心理发展的相关问题，涵盖有关儿童心理发展的对象与任务、儿童心理发展的研究方法与设计、心理发展的基本理论、影响心理发展的遗传与环境因素，以及儿童在认知、智力、语言、情绪、人格、道德等各个领域的发展特点、趋势及影响因素等。


### 加载模型

- 加载文本向量化模型

In [3]:
from BCEmbedding import EmbeddingModel
embedding_model = EmbeddingModel(model_name_or_path="./bce-embedding-base_v1", trust_remote_code=True)

/home/407/anaconda3/envs/llm/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
04/14/2024 12:03:46 - [INFO] -BCEmbedding.models.EmbeddingModel->>>    Loading from `/home/407/bigfile/tangpeng/bce-embedding-base_v1`.
04/14/2024 12:03:47 - [INFO] -BCEmbedding.models.EmbeddingModel->>>    Execute device: cuda;	 gpu num: 4;	 use fp16: False;	 embedding pooling type: cls;	 trust remote code: True


### 对RAG数据库中的问题进行向量化

In [4]:
from tqdm import tqdm
questions = questions.tolist()
vectors = []
for i in tqdm(range(0, len(questions), 512)):
    batch_sens = questions[i: i + 512]
    batch_inputs = embedding_model.encode(batch_sens, enable_tqdm=False)
    vectors.append(batch_inputs)

vectors = np.concatenate(vectors, axis=0)
print(vectors.shape)

100%|██████████| 6/6 [00:08<00:00,  1.40s/it]

(2948, 768)


2948 表示QA数据集大小，有2948个QA对
768 表示将一个问题转化为一个768维的向量表示

### 建立向量索引

In [5]:
# 将向量保存到向量数据库中
import faiss, os
if not os.path.exists("large.index"):
    print("Building index")
    index = faiss.IndexFlatIP(768)              # 创建索引，768是向量的维度
    faiss.normalize_L2(vectors)                 # 对向量进行归一化
    index.add(vectors)                          # 向索引中添加向量
    faiss.write_index(index, "large.index")     # 将索引保存到文件中
else:
    print("Loading index")
    index = faiss.read_index("large.index")     # 从文件中加载索引
index

04/14/2024 12:09:03 - [INFO] -faiss.loader->>>    Loading faiss with AVX2 support.
04/14/2024 12:09:03 - [INFO] -faiss.loader->>>    Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
04/14/2024 12:09:03 - [INFO] -faiss.loader->>>    Loading faiss.
04/14/2024 12:09:03 - [INFO] -faiss.loader->>>    Successfully loaded faiss.


Loading index


<faiss.swigfaiss.IndexFlat; proxy of <Swig Object of type 'faiss::IndexFlat *' at 0x7f608c1e9d40> >

### 对查询进行向量化

In [6]:
quesiton = "最近好烦，有很多事情要做！"
q_vector = embedding_model.encode(quesiton, enable_tqdm=False)
q_vector.shape

(1, 768)

### 召回TOP K

- 根据question向量，从数据库中召回与之最相似的TOP K

In [7]:
faiss.normalize_L2(q_vector)                     
scores, indexes = index.search(q_vector, 10)     
topk_result = np.array(questions)[indexes]       
topk_result = topk_result.tolist()[0]
indexes = indexes.tolist()[0]

question2index = {}
for question, index in zip(topk_result, indexes):
    question2index[question] = index
    
question2index   # 返回的是top10的问题和对应的索引

{'安慰商场里和妈妈走散的小妹妹': 2278,
 '小学生常用什么方式来表达自己的心情？': 163,
 '情绪表达的方式有哪些？': 2428,
 '小学生不良情绪的表现有哪些？': 210,
 '2岁儿童主要使用哪些情绪调节策略？': 162,
 '情绪调节有哪些类型？': 116,
 '青少年通常如何排解性的压力或宣泄内心焦虑与不安？': 1064,
 '情绪调节可以分为哪三大类？': 139,
 '青少年通常如何排解性的压力？': 1065,
 '儿童的基本情绪主要包括哪些？': 2565}

### 召回进行排序

- 使用RerankerModel对召回的结果进行排序，选出最终的TOP 3作为问题回答的上下文

In [9]:
from BCEmbedding import RerankerModel
canidate = topk_result
sentence_pairs = [[quesiton, passage] for passage in canidate]
reranker_model = RerankerModel(model_name_or_path="./bce-reranker-base_v1", trust_remote_code=True)
# scores = model.compute_score(sentence_pairs)
rerank_results = reranker_model.rerank(quesiton, canidate)

context = ""
for i in range(3):
    context += answers[question2index[rerank_results["rerank_passages"][i]]]

context   # 返回的是利用与查询问题最相似的top3问题对应的答案构建的上下文

04/14/2024 12:15:05 - [INFO] -BCEmbedding.models.RerankerModel->>>    Loading from `/home/407/bigfile/tangpeng/bce-reranker-base_v1`.
04/14/2024 12:15:05 - [INFO] -BCEmbedding.models.RerankerModel->>>    Execute device: cuda;	 gpu num: 4;	 use fp16: False
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


'亲社会行为情绪表达的方式主要有言语表情和非言语表情两大类。其中言语表情是通过一个人言语时的音响、音速、音调等变化来反映不同情绪的。而非言语表情又包括面部表情和体态表情两方面。从情绪调节过程的来源划分，可以将情绪调节分为内部调节和外部调节两大类。其中，内部调节主要由个体自身完成，包括对神经生理、认知体验和动作行为的调节。而外部调节则来源于个体以外的环境，如幼儿痛哭时，大人的抚慰可以帮助幼儿尽快地从痛苦中走出来。外部调节可以分为支持性环境调节和破坏性环境调节两大类。'

### 生成回复

In [21]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_dir = "./Shanghai_AI_Laboratory/internlm2-chat-1_8b"
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_dir, trust_remote_code=True, device_map="auto")

# 利用自己微调后的模型进行对话
from peft import PeftModel
model = PeftModel.from_pretrained(model, model_id="./chatbot/checkpoint-876")

model = model.eval()
template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。\n上下文：""" +  context + "\n最后的问题: " + quesiton + "\n有用的回答:"
response, history = model.chat(tokenizer, template, history=[])

print(response)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

你好，我是书生·浦语。
从你的描述来看，你可能需要处理一些比较繁重和复杂的事情，需要你的注意力和精力比较集中，可能也会遇到一些情绪上的困扰，比如烦躁不安等。
你可以尝试给自己列一个清单，列出你需要做的事情，按照优先级安排，这样可以帮助你更好地处理这些事情。
另外，你也可以试着去运动，运动可以释放身体内的压力，缓解情绪上的压力。
希望我的回答能够帮助到你。
祝好（🐳）



- 用作对比，在没有上下文的情况下

In [22]:
template =  quesiton 
response, history = model.chat(tokenizer, template, history=[])
print(response)

你好，我是小九。
最近我也是有很多事情要做，不过我会尝试去分配任务的优先级，先完成最重要的。
有时候我们会遇到一些紧急但并不那么重要的任务，可以先放在后面完成。
当你觉得有些烦躁时，可以先让自己冷静下来，再思考如何安排自己的时间。
祝好！

